In [3]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence="/home01/chenhuangrong/ROC/m6a_data.txt"
RNA_code='ACGU'
k=1
interval=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_data, train_target, test_size=0.1, random_state=0)
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval)] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval)] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval)] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1 :
                for p,c_value in enumerate(code_values):
                    if c_value==line_value[0][j:j+interval]:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1 :
                for p,c_value in enumerate(code_values):
                    if c_value==line_value[0][j:j+interval]:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1 :
                for p,c_value in enumerate(code_values):
                    if c_value==line_value[0][j:j+interval]:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval)] for jj in xrange(len(positive_negative_y_train))]
#     code_values=make_kmer_list(interval,RNA_code)
#     code_len=len(code_values)
#     positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval)] for ii in xrange(code_len)]
#     negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval)] for ii in xrange(code_len)]
#     for i,line_value in enumerate(positive_y_train.values):
#         for j,code_value in enumerate(line_value[0]):
#             if j<= len(line_value[0])-interval-1 :
#                 for p,c_value in enumerate(code_values):
#                     if c_value==line_value[0][j:j+interval]:
#                         positive_seq_value[p][j]+=1
#     positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
#     for i,line_value in enumerate(negative_y_train.values):
#         for j,code_value in enumerate(line_value[0]):
#             if j<= len(line_value[0])-interval-1 :
#                 for p,c_value in enumerate(code_values):
#                     if c_value==line_value[0][j:j+interval]:
#                         negative_seq_value[p][j]+=1
#     negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1 :
                for p,c_value in enumerate(code_values):
                    if c_value==line_value[0][j:j+interval]:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='accuracy')
    clf.fit(X_train, Y_train)
    C=clf.best_params_['C']
    y_pred_prob=clf.predict_proba(X_test)
    
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
#     ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv('/home01/chenhuangrong/ROC/independent_test_%d.csv'%interval,header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save("svm_independent_test",[str(X_train.shape[1])],savedata,'/home01/chenhuangrong/ROC/%d_interval_PSTNP_result_revised.xls'%interval)


    
        



    

c: 9.09570324078 gamma: 4.0
0.763358778626
ACC_all: 0.763358778626
c: 6.34960420787 gamma: 1.62868059835
0.770992366412
ACC_all: 1.53435114504
c: 3.09433550365 gamma: 1.62868059835
0.793893129771
ACC_all: 2.32824427481
c: 18.6644646332 gamma: 0.323171161892
0.759541984733
ACC_all: 3.08778625954
c: 13.0294447868 gamma: 0.22560201395
0.687022900763
ACC_all: 3.77480916031
c: 10.8863200014 gamma: 0.386791937957
0.774809160305
ACC_all: 4.54961832061
c: 7.59961285667 gamma: 0.462937356144
0.774809160305
ACC_all: 5.32442748092
c: 18.6644646332 gamma: 1.94930921824
0.734615384615
ACC_all: 6.05904286553
c: 15.594473746 gamma: 0.386791937957
0.696153846154
ACC_all: 6.75519671169
c: 6.34960420787 gamma: 0.462937356144
0.673076923077
ACC_all: 7.42827363476
[[['svmC:6.34960420787gamma:0.462937356144', 0.74282736347621847, 0.7392827383789669, 0.7489078097475045, 0.7489078097475045, 0.7367469172049325, 0.7425918417241492, 0.74383596985056522, 0.74383596985056522, 0.48601396961827692, 0.78934452764204

True

In [208]:
negative_seq_value,positive_seq_value

(matrix([[ 0.        ,  0.        ,  0.        , ...,  0.00229533,
           0.00153022,  0.        ],
         [ 0.        ,  0.00076511,  0.        , ...,  0.        ,
           0.        ,  0.00076511],
         [ 0.        ,  0.00076511,  0.        , ...,  0.        ,
           0.00076511,  0.        ],
         ..., 
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.00076511,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.00076511,  0.        ],
         [ 0.        ,  0.        ,  0.00076511, ...,  0.00076511,
           0.00153022,  0.00153022]]),
 matrix([[ 0.        ,  0.        ,  0.00076511, ...,  0.00076511,
           0.00076511,  0.00076511],
         [ 0.        ,  0.00076511,  0.        , ...,  0.        ,
           0.        ,  0.00076511],
         [ 0.        ,  0.        ,  0.00076511, ...,  0.        ,
           0.00076511,  0.        ],
         ..., 
         [ 0.        ,  0.      

In [176]:
# print ACC_all

# print list(np.array(y_pred_prob_all).astype(list)[:,1])
# print list(np.array(Y_all).T)
# all_y=[list(np.array(Y_all).T),list(np.array(y_pred_all).T),list(np.array(y_pred_prob_all).astype(list)[:,1])]
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv('/home01/chenhuangrong/ROC/independent_test_%d.csv'%interval,header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc/divided_num,TP_all/divided_num,
            FN_all/divided_num,FP_all/divided_num,TN_all/divided_num,pos_all/divided_num,neg_all/divided_num]]]
print savedata
easy_excel.save("svm_independent_test",[str(X_train.shape[1])],savedata,'/home01/chenhuangrong/ROC/%d_interval_PSTNP_result22222.xls'%interval)


[[['svmC:26.736537569gamma:0.462937356144', 0.060769230769230763, 0.7378091612272641, 0.3, 0.3, 0.9099177921315327, 0.341330162033458, 0.30823155870009999, 0.30823155870009999, 0.27177446522269416, 0.080607979281708902, 39.2, 91.5, 11.8, 118.9, 130.7, 130.7]]]


True

In [ ]:
SN_all

In [67]:
## kf = KFold(n_splits=5,shuffle=False)  
for train_index,test_index in kf.split(positive_seq[100]):
    print train_index
    print test_index
# import numpy as np
# c=[3,4]
# d=[5,6]
# p=[c,d]
# p=np.matrix(p).T
# print p
# pd.DataFrame(p).to_csv('test.csv',header=None,index=False)
np.array([[1,2,3],[4,5,6]]).shape[0]

[11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10 22 23 24 25 26 27 28 29 30 31 32 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]
[11 12 13 14 15 16 17 18 19 20 21]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]
[22 23 24 25 26 27 28 29 30 31]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 42 43 44 45 46 47 48 49 50 51]
[32 33 34 35 36 37 38 39 40 41]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41]
[42 43 44 45 46 47 48 49 50 51]


2